<a href="https://colab.research.google.com/github/leejunho12316/HonGongMachine/blob/main/OpenAI_Assistant_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#환경설정

In [1]:
!pip install openai

In [2]:
import openai
OPENAI_API_KEY = None
client = openai.OpenAI(api_key = OPENAI_API_KEY)

In [3]:
!wget https://github.com/chatgpt-kr/openai-api-tutorial/raw/main/ch11/%EC%B6%95%EA%B5%AC%EA%B7%9C%EC%B9%99%EC%A0%95%EB%A6%AC.pdf

--2026-02-19 06:24:49--  https://github.com/chatgpt-kr/openai-api-tutorial/raw/main/ch11/%EC%B6%95%EA%B5%AC%EA%B7%9C%EC%B9%99%EC%A0%95%EB%A6%AC.pdf
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/chatgpt-kr/openai-api-tutorial/main/ch11/%EC%B6%95%EA%B5%AC%EA%B7%9C%EC%B9%99%EC%A0%95%EB%A6%AC.pdf [following]
--2026-02-19 06:24:49--  https://raw.githubusercontent.com/chatgpt-kr/openai-api-tutorial/main/ch11/%EC%B6%95%EA%B5%AC%EA%B7%9C%EC%B9%99%EC%A0%95%EB%A6%AC.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 22073717 (21M) [application/octet-stream]
Saving to: ‘축구규칙정리.pdf’

축구규칙정리.pdf    100%[=

#OpenAI Assistant API

VectorDB, File Upload

In [4]:
#벡터 저장소 생성
vector_store = client.vector_stores.create(name = '축구 규칙 파일')
vector_store

VectorStore(id='vs_6996accbce148191be578e15f025018b', created_at=1771482316, file_counts=FileCounts(cancelled=0, completed=0, failed=0, in_progress=0, total=0), last_active_at=1771482316, metadata={}, name='축구 규칙 파일', object='vector_store', status='completed', usage_bytes=0, expires_after=None, expires_at=None, description=None)

In [5]:
#파일 불러오기
file_streams = open('축구규칙정리.pdf', 'rb')
file_streams

<_io.BufferedReader name='축구규칙정리.pdf'>

In [6]:
#파일 업로드
file_batch = client.vector_stores.file_batches.upload_and_poll(
    vector_store_id = vector_store.id,
    files = [file_streams]
)

In [7]:
print(file_batch.status)
print(file_batch.file_counts)

completed
FileCounts(cancelled=0, completed=1, failed=0, in_progress=0, total=1)


In [8]:
failed_files = client.vector_stores.files.list(
    vector_store_id = vector_store.id,
    filter='failed'
)

for f in failed_files:
  print(f)

Create Assistant

In [9]:
instruction = '''
[목적]
이 GPT는 축구 규칙을 상세히 설명해주는 챗봇입니다.

[규칙]
1. 사용자가 축구 규칙에 대해 질문하면 업로드된 파일에서 해당 내용을 찾아 자세히 답변합니다.
2. 파일안에서 마땅한 답을 찾을 수 없거나 축구 규칙에 관한 질문이 아니면 "축구 규칙에 관한 질문만 부탁해요^^" 라고 답해주세요.
3. 답변의 형태는 아래 예시와 같이 해주세요

예시)
-  질문 : 질문 내용
-  답변 : 답변 내용
4. 모든 질문에 한국어로 답변해주세요.
'''

assistant = client.beta.assistants.create(
    name = "축구 규칙 설명 챗봇", #assistant 이름
    instructions = instruction,   #챗봇 규칙, 설정
    model = 'gpt-4o',
    #기능 code_interpreter : 코드 인터프리터, file_search : 파일 참고, function : 펑션 콜링
    tools = [{'type' : 'file_search'}],
    #코드 인터프리터 : {'code_interpreter' : {'file_ids' : [file.id]}}
    tool_resources = {'file_search' : {'vector_store_ids' : [vector_store.id]}}
)

print(assistant.id)

/tmp/ipython-input-1766267036.py:16: DeprecationWarning: deprecated
  assistant = client.beta.assistants.create(


asst_7p3Nu2h04HLQtfBO9C8rlu7P


In [10]:
assistant

Assistant(id='asst_7p3Nu2h04HLQtfBO9C8rlu7P', created_at=1771482415, description=None, instructions='\n[목적]\n이 GPT는 축구 규칙을 상세히 설명해주는 챗봇입니다.\n\n[규칙]\n1. 사용자가 축구 규칙에 대해 질문하면 업로드된 파일에서 해당 내용을 찾아 자세히 답변합니다.\n2. 파일안에서 마땅한 답을 찾을 수 없거나 축구 규칙에 관한 질문이 아니면 "축구 규칙에 관한 질문만 부탁해요^^" 라고 답해주세요.\n3. 답변의 형태는 아래 예시와 같이 해주세요\n\n예시)\n-  질문 : 질문 내용\n-  답변 : 답변 내용\n4. 모든 질문에 한국어로 답변해주세요.\n', metadata={}, model='gpt-4o', name='축구 규칙 설명 챗봇', object='assistant', tools=[FileSearchTool(type='file_search', file_search=FileSearch(max_num_results=None, ranking_options=FileSearchRankingOptions(score_threshold=0.0, ranker='default_2024_08_21', hybrid_search=None)))], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=None, file_search=ToolResourcesFileSearch(vector_store_ids=['vs_6996accbce148191be578e15f025018b'])), top_p=1.0, reasoning_effort=None)

Modify Assistant

In [57]:
instruction = '''
[목적]
이 GPT는 축구 규칙을 상세히 설명해주는 챗봇입니다.

[규칙]
1. 사용자가 축구 규칙에 대해 질문하면 업로드된 파일에서 해당 내용을 찾아 자세히 답변합니다.
2. 파일안에서 마땅한 답을 찾을 수 없거나 축구 규칙에 관한 질문이 아니면 "축구 규칙에 관한 질문만 부탁해요^^" 라고 답해주세요.
3. 답변의 형태는 아래 예시와 같이 해주세요

예시)
-  질문 : 질문 내용
-  답변 : 답변 내용
4. 모든 질문에 한국어로 답변해주세요.
'''

In [58]:
assitant = client.beta.assistants.update(
    assistant.id,
    instructions = instruction,
    model = 'gpt-4o',
    tools = [{'type' : 'file_search'}],
    tool_resources = {'file_search' : {'vector_store_ids' : [vector_store.id]}}
)

/tmp/ipython-input-481585206.py:1: DeprecationWarning: deprecated
  assitant = client.beta.assistants.update(


Assistants List

In [11]:
my_assistants = client.beta.assistants.list(order = 'desc', limit=10)
print(my_assistants)
print(my_assistants.data[0].id)

/tmp/ipython-input-494306868.py:1: DeprecationWarning: deprecated
  my_assistants = client.beta.assistants.list(order = 'desc', limit=10)


SyncCursorPage[Assistant](data=[Assistant(id='asst_7p3Nu2h04HLQtfBO9C8rlu7P', created_at=1771482415, description=None, instructions='\n[목적]\n이 GPT는 축구 규칙을 상세히 설명해주는 챗봇입니다.\n\n[규칙]\n1. 사용자가 축구 규칙에 대해 질문하면 업로드된 파일에서 해당 내용을 찾아 자세히 답변합니다.\n2. 파일안에서 마땅한 답을 찾을 수 없거나 축구 규칙에 관한 질문이 아니면 "축구 규칙에 관한 질문만 부탁해요^^" 라고 답해주세요.\n3. 답변의 형태는 아래 예시와 같이 해주세요\n\n예시)\n-  질문 : 질문 내용\n-  답변 : 답변 내용\n4. 모든 질문에 한국어로 답변해주세요.\n', metadata={}, model='gpt-4o', name='축구 규칙 설명 챗봇', object='assistant', tools=[FileSearchTool(type='file_search', file_search=FileSearch(max_num_results=None, ranking_options=FileSearchRankingOptions(score_threshold=0.0, ranker='default_2024_08_21', hybrid_search=None)))], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=None, file_search=ToolResourcesFileSearch(vector_store_ids=['vs_6996accbce148191be578e15f025018b'])), top_p=1.0, reasoning_effort=None)], has_more=False, object='list', first_id='asst_7p3Nu2h04HLQtfBO9C8rlu7P', last_id='asst_7p3Nu2h04HLQt

In [12]:
assistant = client.beta.assistants.retrieve(my_assistants.data[0].id)
assistant

/tmp/ipython-input-2131993979.py:1: DeprecationWarning: deprecated
  assistant = client.beta.assistants.retrieve(my_assistants.data[0].id)


Assistant(id='asst_7p3Nu2h04HLQtfBO9C8rlu7P', created_at=1771482415, description=None, instructions='\n[목적]\n이 GPT는 축구 규칙을 상세히 설명해주는 챗봇입니다.\n\n[규칙]\n1. 사용자가 축구 규칙에 대해 질문하면 업로드된 파일에서 해당 내용을 찾아 자세히 답변합니다.\n2. 파일안에서 마땅한 답을 찾을 수 없거나 축구 규칙에 관한 질문이 아니면 "축구 규칙에 관한 질문만 부탁해요^^" 라고 답해주세요.\n3. 답변의 형태는 아래 예시와 같이 해주세요\n\n예시)\n-  질문 : 질문 내용\n-  답변 : 답변 내용\n4. 모든 질문에 한국어로 답변해주세요.\n', metadata={}, model='gpt-4o', name='축구 규칙 설명 챗봇', object='assistant', tools=[FileSearchTool(type='file_search', file_search=FileSearch(max_num_results=None, ranking_options=FileSearchRankingOptions(score_threshold=0.0, ranker='default_2024_08_21', hybrid_search=None)))], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=None, file_search=ToolResourcesFileSearch(vector_store_ids=['vs_6996accbce148191be578e15f025018b'])), top_p=1.0, reasoning_effort=None)

#Thread 생성

In [45]:
thread1 = client.beta.threads.create()
print(thread1)

/tmp/ipython-input-1435491537.py:1: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  thread1 = client.beta.threads.create()


Thread(id='thread_sURAZdy4ROUGHa4b7bV4W8wF', created_at=1771483596, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))


In [46]:
message = client.beta.threads.messages.create(
    thread_id = thread1.id,
    role = 'user',
    content = '축구장이 무엇인가?'
)
print(message)

/tmp/ipython-input-2727508681.py:1: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  message = client.beta.threads.messages.create(


Message(id='msg_HCNywJigy0NQAGjMLn7AKo23', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='축구장이 무엇인가?'), type='text')], created_at=1771483604, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_sURAZdy4ROUGHa4b7bV4W8wF')


#Run

Thread 내에서 Assistant 실행

In [47]:
#run : 실행 상태가 담김 (completed, queued, failed 등등)
run = client.beta.threads.runs.create(
    thread_id = thread1.id,
    assistant_id = assistant.id
)
run.status

/tmp/ipython-input-3962873598.py:2: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.create(


'queued'

retrieve() 메서드로 질문 실행

In [48]:
import time

while run.status not in ['completed','failed']:
  run = client.beta.threads.runs.retrieve(
      thread_id = thread1.id,
      run_id = run.id
  )
  print(run.status)
  time.sleep(1)

/tmp/ipython-input-2082174778.py:4: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.retrieve(


completed


메세지 내역 출력

In [49]:
conversations = client.beta.threads.messages.list(thread_id = thread1.id)
for c in conversations:
  print(c.role)
  print(c.content[0].text.value)
  print()

assistant
- 질문: 축구장이 무엇인가?
- 답변: 축구장은 축구 경기가 이루어지는 장소로, 일반적으로 직사각형 형태의 필드입니다. 축구장은 경계 라인으로 경계를 이루며, 각 골대가 위치한 골라인, 터치라인, 그리고 필드의 중앙을 가로지르는 중앙선으로 구분됩니다. 

축구장의 크기는 국제 축구 규정(IFAB)에 의해 제정되며, 공식 경기에 사용되는 경기장의 크기는 폭 45m에서 90m, 길이 90m에서 120m 내에서 다양할 수 있습니다. 각 경기장은 주어진 공간과 시설에 따라 이 규정 내에서 최적의 크기를 채택합니다【4:5†축구규칙정리.pdf】.

user
축구장이 무엇인가?



/tmp/ipython-input-4177141907.py:1: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  conversations = client.beta.threads.messages.list(thread_id = thread1.id)


#정리

In [ ]:
#쓰레드
thread1 = client.beta.threads.create()
print(thread1)

In [56]:
#쓰레드에 메세지 추가
message = client.beta.threads.messages.create(
    thread_id = thread1.id,
    role = 'user',
    content = '규칙에서 심판은 사람으로 이루어져 있는가 동물로 이루어져 있는가?'
)

#질문 대기 상태 (queued)
run = client.beta.threads.runs.create(
    thread_id = thread1.id,
    assistant_id = assistant.id
)

#질문 실행
while run.status not in ['completed','failed']:
  run = client.beta.threads.runs.retrieve(
      thread_id = thread1.id,
      run_id = run.id
  )
  print(run.status)
  time.sleep(1)

#대화 출력
conversations = client.beta.threads.messages.list(thread_id = thread1.id)
for c in conversations:
  print(c.role)
  print(c.content[0].text.value)
  print()

/tmp/ipython-input-3536579927.py:2: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  message = client.beta.threads.messages.create(
/tmp/ipython-input-3536579927.py:9: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.create(
/tmp/ipython-input-3536579927.py:16: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.retrieve(


queued
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
completed
assistant
- 질문: 규칙에서 심판은 사람으로 이루어져 있는가 동물로 이루어져 있는가?
- 답변: 축구 규칙에서는 심판은 사람으로 구성됩니다. 경기 중 주심과 다른 심판들은 사람으로, 경기의 규칙을 적용하고 공정한 진행을 보장하기 위해 여러 의무를 수행합니다【4:0†축구규칙정리.pdf】.

user
규칙에서 심판은 사람으로 이루어져 있는가 동물로 이루어져 있는가?

assistant
축구 규칙에 관한 질문만 부탁해요^^

user
심판은 사람인가 동물인가?

assistant
축구 규칙에 관한 질문만 부탁해요^^

user
심판은 사람인가 동물인가?

assistant
- 질문: 축구장은 왜 직사각형인가?
- 답변: 축구장이 직사각형인 이유는 역사적으로 경기의 공정성과 명확한 규칙을 유지하기 위해 지정된 것입니다. 직사각형 형태는 각 팀이 공격과 수비를 균등하게 펼 칠 수 있도록 합니다. 또한, 경계선이 명확하게 정의되기 때문에 경기 진행이 명확해지고 판정의 명확성을 높입니다. 직사각형 형태의 경기장은 현대 축구의 규칙에 들어와 공인된 표준 형태가 되었으며, 이는 경기의 질과 체계를 유지하는 데 중요합니다.

user
축구장은 왜 직사각형인가?

user
축구장은 왜 직사각형인가?

assistant
- 질문: 축구장이 무엇인가?
- 답변: 축구장은 축구 경기가 이루어지는 장소로, 일반적으로 직사각형 형태의 필드입니다. 축구장은 경계 라인으로 경계를 이루며, 각 골대가 위치한 골라인, 터치라인, 그리고 필드의 중앙을 가로지르는 중앙선으로 구분됩니다. 

축구장의 크기는 국제 축구 규정(IFAB)에 의해 제정되며, 공식 경기에 사용되는 경기장의 크기는 폭 45m에서 90m, 길이 90m에서 120m 내에서 다양할 수 있습니다. 각 경기장은 주어진 공간과 시설에 따라 이 규정 내에서 최적의 크기를 채

/tmp/ipython-input-3536579927.py:24: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  conversations = client.beta.threads.messages.list(thread_id = thread1.id)


In [51]:
run.status

'completed'

#삭제

In [57]:
response = client.beta.assistants.delete(assistant.id)
print(response)

/tmp/ipython-input-4275888659.py:1: DeprecationWarning: deprecated
  response = client.beta.assistants.delete(assistant.id)


AssistantDeleted(id='asst_7p3Nu2h04HLQtfBO9C8rlu7P', deleted=True, object='assistant.deleted')


In [60]:
file_list = client.files.list()
for f in file_list:
  print(f.id)

response = client.files.delete(f.id)
print(response)

file-Y2iYvks2cXtrxciJD9CrEM
FileDeleted(id='file-Y2iYvks2cXtrxciJD9CrEM', deleted=True, object='file')
